In [1]:
from pymilvus import MilvusClient
from typing import Optional
import shutil
import json
import numpy as np
from uuid import uuid4
import os
import random

In [ ]:
# remove data folder if it exists
if "data" in os.listdir():
    shutil.rmtree("data")
    
os.mkdir("data")

In [ ]:
DB_PATH = "data/milvus_demo.db"
TABLE_NAME = "test_table" 
DIM_VALUE = 10  # embedding dimension

### Create Database and Tables

In [4]:
client = MilvusClient(uri=DB_PATH)

In [ ]:
# remove table if it exists
if client.has_collection(collection_name=TABLE_NAME):
    client.drop_collection(collection_name=TABLE_NAME)
    

client.create_collection(
    collection_name=TABLE_NAME,
    dimension=DIM_VALUE,
)

### Embedding

In [6]:
class TextEmbeddingModel:
    def __init__(self, model_name_or_path: Optional[str] = None) -> None:
        pass
    
    def encode_text(self, text: str) -> list:
        return np.random.rand(DIM_VALUE).tolist()

In [7]:
text_embedding_model = TextEmbeddingModel()
text_embedding_model.encode_text(text="hello world")

[0.6356646235284901,
 0.8070105865996785,
 0.3380619137061106,
 0.3262281970361789,
 0.9328314501186383,
 0.04234539804552884,
 0.705254584305317,
 0.5708579569436718,
 0.05624048570081808,
 0.28315828975112567]

### Generate Fake Data

In [8]:
def build_fake_data() -> list:
    random_text = [
        "hello world",
        "good night",
        "python is the best language"
    ] * 100
    random_text = [i + str(uuid4()) for i in random_text]
    data = []
    for i, text in enumerate(random_text):
        data.append(
            {
                "id": i,
                "text": text,
                "vector": text_embedding_model.encode_text(text=text),
                "class_v1": random.choice(seq=["A", "B", "C"]),
                "extra_data": json.dumps(
                    {"key1": random.randint(a=1, b=100), "key2": random.randint(a=1, b=100)},
                    ensure_ascii=False
                ),
            }
        )
    return data

In [9]:
fake_data = build_fake_data()

In [10]:
fake_data[0]

{'id': 0,
 'text': 'hello world8415f267-5ad8-47f7-823d-a6a2853110af',
 'vector': [0.03298326729718426,
  0.9582191303164717,
  0.8709208965526758,
  0.7481400353058123,
  0.9817916609659837,
  0.40061512525935583,
  0.31489820010031644,
  0.3730439929914975,
  0.25418375765483014,
  0.9980153833213683],
 'class_v1': 'C',
 'extra_data': '{"key1": 76, "key2": 14}'}

### Add Data to Table

In [11]:
res = client.insert(collection_name=TABLE_NAME, data=fake_data)

res

{'insert_count': 300, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

### Vector Search

In [12]:
query_text = "我想查找这个文本"
query_vector = text_embedding_model.encode_text(query_text)

res = client.search(
    collection_name=TABLE_NAME,
    data=[query_vector],
    top_k=5,
    output_fields=['id', 'text', 'class_v1'],
    filter="class_v1 in ['A', 'B']",
)

In [13]:
res[0]

[{'id': 107,
  'distance': 0.9499340057373047,
  'entity': {'text': 'python is the best languaged405c467-6df8-4e3f-9b86-7877712b906b',
   'class_v1': 'A',
   'id': 107}},
 {'id': 220,
  'distance': 0.9470595717430115,
  'entity': {'text': 'good nightb62bbebf-7969-4337-8e79-1a05ae12f64f',
   'class_v1': 'A',
   'id': 220}},
 {'id': 201,
  'distance': 0.9138093590736389,
  'entity': {'text': 'hello world528174a5-9313-4504-b75f-127d93bce57d',
   'class_v1': 'B',
   'id': 201}},
 {'id': 34,
  'distance': 0.9051440954208374,
  'entity': {'text': 'good nightfe1e3607-2e99-49b6-a0d2-3e1be1d26052',
   'class_v1': 'B',
   'id': 34}},
 {'id': 70,
  'distance': 0.9020692706108093,
  'entity': {'text': 'good nightdb1ac57d-a875-4882-a905-59dfb0757fef',
   'class_v1': 'B',
   'id': 70}},
 {'id': 265,
  'distance': 0.9014686346054077,
  'entity': {'text': 'good nightb1beaea0-2ff7-4867-bd8f-5e681f822f3a',
   'class_v1': 'B',
   'id': 265}},
 {'id': 209,
  'distance': 0.9010605812072754,
  'entity': {'t

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("maidalun1020/bce-embedding-base_v1")